# Baseline Models
- LDA
- STM

## Data

In [34]:
path = '../data/'

In [35]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn import metrics
from gensim.models import CoherenceModel
from sklearn.metrics import classification_report
import gensim
from gensim.models.ldamodel import LdaModel

def q_metrics(y_true, y_pred,my_model=None):
    contigency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    purity = np.sum(np.amax(contigency_matrix, axis=0)) / np.sum(contigency_matrix)
  
    print('purity_score:',purity)
    print('NMI:',metrics.normalized_mutual_info_score(y_true, y_pred))
    
    if my_model!=None:
        cm = CoherenceModel(model=my_model, corpus=bow_corpus, dictionary=dictionary, coherence='u_mass')
        print('Coherence:',cm.get_coherence())

def print_result(resres,my_model=None):
    pred = []
    for i in range(len(resres)):
        pred.append(np.argmax(resres[i]))
  
    y_true = df[df['div']=='test']['topic']
    y_pred = pred
    q_metrics(y_true, y_pred,my_model)
  

## LDA

In [36]:
d_name='insurance'

In [37]:
# df=pd.read_csv(path+'total_preprocessed_data.csv')
df=pd.read_csv(f'{path}df_pre_with_hashtags_{d_name}.csv')
print('-- Dataframe shape: {}'.format(df.shape))
df.dropna(subset='pre_text',inplace=True)
df = df.drop_duplicates(subset='pre_text')
df['topic']='insurance'
df = df[df['topic']==d_name]
# df = df.groupby('topic').apply(lambda x: x.sample( len(x) if len(x) < 10000 else 20000,random_state=42 )).reset_index(drop=True)
df = df.reset_index(drop=True)

print(Counter(df.topic))
print(df.shape)

label_change = {'بازنشسته تعمین اجتماعی': 'تعمین اجتماعی',
         'بیمه خودرو': 'شخص ثالث', 
         'بیمه شخص ثالث': 'شخص ثالث',
         'بیمه تکمیلی': 'بیمه',
         'بیمه برنده مالی': 'بیمه',
         'بیمه دولتی': 'بیمه',
         'کارمند بیمه': 'بیمه',
         'بیمه خصوصی': 'بیمه',
         'بیمه در بورس': 'بیمه',
         'بیمه عمر': 'تعمین اجتماعی',
         'بیمه بیکاری': 'تعمین اجتماعی',
         'بیمه پسرا': 'بیمه پزشکی',
         'بیمه بازنشستگی': 'تعمین اجتماعی'}

df_labeled = pd.read_csv(f"{path}/labeled_sample_{d_name}.csv")
df_labeled.dropna(subset='label',inplace=True)
df_labeled = df_labeled[df_labeled.label!='-']
print(Counter(df_labeled.label))

df_labeled['label']=df_labeled['label'].apply(lambda x:label_change[x] if x in label_change.keys() else x)

label_change = {}
for k,v in zip(['شخص ثالث', 'بیمه', 'تعمین اجتماعی', 'بیمه پزشکی' ],['person','insurance','tamin','health']):
    label_change[k]=v
    
df_labeled['label']=df_labeled['label'].apply(lambda x:label_change[x] if x in label_change.keys() else x)
    
print('\n\nTest data labels:',Counter(df_labeled.label))


df = pd.merge(df,df_labeled[['text','label']],how='left',on='text',indicator=True)
df=df[df['_merge']=='left_only']
df = df[df['is_relevant']==1]
# df=df.sample(n=df.shape[0] if len(x) < 10000 else 20*1000,replace=False,random_state=42,ignore_index=True)

df_labeled['div']='test'
df['div']='train'
df=pd.concat([df,df_labeled],ignore_index=True)

print('train and test data volum:',Counter(df['div']))

-- Dataframe shape: (27880, 59)
Counter({'insurance': 26294})
(26294, 60)
Counter({'بیمه': 20, 'بیمه پزشکی': 19, 'بازنشسته تعمین اجتماعی': 15, 'بیمه خودرو': 10, 'تعمین اجتماعی': 9, 'شخص ثالث': 4, 'بیمه شخص ثالث': 3, 'بیمه تکمیلی': 3, 'بیمه بیکاری': 3, 'بیمه بازنشستگی': 2, 'بیمه عمر': 2, 'بیمه دولتی': 2, 'بیمه برنده مالی': 1, 'کارمند بیمه': 1, 'بیمه پسرا': 1, 'بیمه خصوصی': 1, 'بیمه در بورس': 1})


Test data labels: Counter({'tamin': 31, 'insurance': 29, 'health': 20, 'person': 17})
train and test data volum: Counter({'train': 19224, 'test': 97})


In [38]:
df

,Unnamed: 0,id,status_id,created_at,text,reply_count,seen_date,is_quote,is_retweet,quote_count,...,stance,pre_text,pre_text_noHAshtag,pre_text_wHashtag,topic,label,_merge,div,pred_topic,tok
0,2,2761968,1.635129e+18,2023-03-13 04:01:53.000000,خب دوستان به احتمالی آلبوم کای رو برای چنل یوت...,NaN,2023-03-13 08:00:05.000000,0,1,NaN,...,1.0,دوستان احتمالی آلبوم کای چنل یوتیوبم ری اکت بر...,دوستان احتمالی آلبوم کای چنل یوتیوبم ری اکت بر...,دوستان احتمالی آلبوم کای چنل یوتیوبم ری اکت بر...,insurance,NaN,left_only,train,NaN,NaN
1,105,2762198,1.635188e+18,2023-03-13 07:56:55.000000,@_Asa2002 رسمی برو، سرت بالا بگیر، اگر نمی‌دون...,NaN,2023-03-15 08:02:27.000000,0,0,NaN,...,2.0,رسمی برو سرت دونستی جواب بلد مطالعه سرچ میزنم ...,رسمی برو سرت دونستی جواب بلد مطالعه سرچ میزنم ...,رسمی برو سرت دونستی جواب بلد مطالعه سرچ میزنم ...,insurance,NaN,left_only,train,NaN,NaN
2,106,2762199,1.635187e+18,2023-03-13 07:53:03.000000,موعد تمدید بیمه بدنه ماشینم شده و با هیچ کدوم‌...,NaN,2023-03-13 20:24:18.000000,0,0,NaN,...,1.0,موعد تمدید بیمه بدنه ماشینم کدوم تون میل سخنم,موعد تمدید بیمه بدنه ماشینم کدوم تون میل سخنم,موعد تمدید بیمه بدنه ماشینم کدوم تون میل سخنم,insurance,NaN,left_only,train,NaN,NaN
3,107,2762200,1.635186e+18,2023-03-13 07:47:00.000000,@nadia_badsun ذاتا دندونپزشکا با بیمه کاری ندا...,NaN,2023-03-17 04:23:03.000000,0,0,NaN,...,0.0,ذاتا دندونپزشکا بیمه کاری ندارن هربار مجبورم خ...,ذاتا دندونپزشکا بیمه کاری ندارن هربار مجبورم خ...,ذاتا دندونپزشکا بیمه کاری ندارن هربار مجبورم خ...,insurance,NaN,left_only,train,NaN,NaN
4,109,2762202,1.635183e+18,2023-03-13 07:34:57.000000,بیمه معوقات اسفند سال پیش و فروردین و اردیبهشت...,NaN,2023-03-16 08:37:06.000000,0,0,NaN,...,0.0,بیمه معوقات اسفند سال فروردین اردیبهشت تاخیری ...,بیمه معوقات اسفند سال فروردین اردیبهشت تاخیری ...,بیمه معوقات اسفند سال فروردین اردیبهشت تاخیری ...,insurance,NaN,left_only,train,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19316,629,2839662,1.640000e+18,38:40.0,@kavaluo چون که مالید، ماشینشم عرف جامعه نیست ...,NaN,04:03.0,0,0,NaN,...,2.0,مالید ماشینشم عرف جامعه بیمه پولشو نمیده,مالید ماشینشم عرف جامعه بیمه پولشو نمیده,مالید ماشینشم عرف جامعه بیمه پولشو نمیده,insurance,person,NaN,test,0,"['مالید', 'ماشینشم', 'عرف', 'جامعه', 'بیمه', '..."
19317,653,2995488,1.640000e+18,47:25.0,@Alireza_George نهه بابا بیمه کجا بود؟؟\nدوزار...,NaN,03:39.0,0,0,NaN,...,2.0,نهه بابا بیمه دوزاری اینجانب کج میباشد,نهه بابا بیمه دوزاری اینجانب کج میباشد,نهه بابا بیمه دوزاری اینجانب کج میباشد,insurance,insurance,NaN,test,0,"['نهه', 'بابا', 'بیمه', 'دوزاری', 'اینجانب', '..."
19318,661,4567723,1.660000e+18,42:33.0,@DonMorteza3 @faryadbiseda5 @Alirezapaknej بیم...,NaN,03:42.0,0,0,NaN,...,0.0,بیمه بیکاری بیمه تامین اجتماعی بقیه درمان دونم...,بیمه بیکاری بیمه تامین اجتماعی بقیه درمان دونم...,بیمه بیکاری بیمه تامین اجتماعی بقیه درمان دونم...,insurance,tamin,NaN,test,0,"['بیمه', 'بیکاری', 'بیمه', 'تامین', 'اجتماعی',..."
19319,665,4626950,1.660000e+18,04:31.0,@fatemenajafi10 پ.ن: یاد مدل هزینه بیمه تو ایر...,NaN,10:21.0,0,0,NaN,...,0.0,یاد مدل هزینه بیمه ایران افتادم,یاد مدل هزینه بیمه ایران افتادم,یاد مدل هزینه بیمه ایران افتادم,insurance,insurance,NaN,test,0,"['یاد', 'مدل', 'هزینه', 'بیمه', 'ایران', 'افتا..."


In [12]:
# df[['topic',
#        'text', 'pre_text',
#        'label',
#        'div', 'pred_topic']].to_csv(f'{path}{d_name}_dataset2.csv')

In [13]:
# print(Counter(df_labeled.label).keys())
# ['سایر غیر کالایی', 'بورس', 'ارز', 'کالا', 'نفت', 'ارز دیجیتال']
# ['other','stok','currency','good','currency','currency']

In [39]:
# df.to_csv(f'{path}{d_name}_dataset.csv')

In [15]:

df['tok'] = df['pre_text'].apply(lambda x: set(x.split()))

train_docs = df[df['div']=='train'].tok.to_numpy()
dictionary = gensim.corpora.Dictionary(train_docs)
bow_corpus = [dictionary.doc2bow(doc) for doc in df.tok]
df['corpus']=bow_corpus


In [29]:
def calculate_coherence_score(i, alpha, beta):
    lda_result=LdaModel(corpus=df[df['div']=='train']['corpus'], id2word=dictionary,
               iterations=i , num_topics=TOPICS, 
               chunksize=2000, random_state=42, gamma_threshold=0.001,
               passes=10, update_every=1,
               alpha=alpha,eta = beta)

    test_corpus_bow = df[df['div']=='test']['corpus'].to_numpy()
    test_res = lda_result[test_corpus_bow]

    pred=[]
    for x in test_res:
        x={k[0]:k[1] for k in x}
        pred.append(max(x,key=x.get) )

    y_true = df[df['div']=='test']['label'] 
    y_pred = pred
    q_metrics(y_true, y_pred)    
    
    return coherence_lda

#list containing various hyperparameters
no_of_iteration = [10,30]
alpha_list = ['symmetric',0.4,0.7]
beta_list = ['auto',0.4,0.7]


for i in no_of_iteration:
    for alpha in alpha_list:
        for beta in beta_list:
            calculate_coherence_score(i, alpha, beta)
            print(f"i : {i} ; alpha : {alpha} ; beta : {beta} ")



purity_score: 0.5360824742268041
NMI: 0.3087885274893255
i : 10 ; alpha : symmetric ; beta : auto 
purity_score: 0.4742268041237113
NMI: 0.2732203531768691
i : 10 ; alpha : symmetric ; beta : 0.4 
purity_score: 0.4639175257731959
NMI: 0.2556468850923897
i : 10 ; alpha : symmetric ; beta : 0.7 
purity_score: 0.5463917525773195
NMI: 0.26752984076139996
i : 10 ; alpha : 0.4 ; beta : auto 
purity_score: 0.5051546391752577
NMI: 0.27810469640964913
i : 10 ; alpha : 0.4 ; beta : 0.4 
purity_score: 0.4845360824742268
NMI: 0.28066165038118557
i : 10 ; alpha : 0.4 ; beta : 0.7 
purity_score: 0.5670103092783505
NMI: 0.25435076872717727
i : 10 ; alpha : 0.7 ; beta : auto 
purity_score: 0.5670103092783505
NMI: 0.32553015391056067
i : 10 ; alpha : 0.7 ; beta : 0.4 
purity_score: 0.4536082474226804
NMI: 0.2595919332728681
i : 10 ; alpha : 0.7 ; beta : 0.7 
purity_score: 0.5463917525773195
NMI: 0.3200284180191951
i : 30 ; alpha : symmetric ; beta : auto 
purity_score: 0.4536082474226804
NMI: 0.2497324

In [27]:
TOPICS = 8

In [30]:
i = 30 ; alpha = 0.7 ; beta = 'auto'

lda_result=LdaModel(corpus=df[(df['div']=='train')]['corpus'], id2word=dictionary,
               iterations=i , num_topics=TOPICS,
               chunksize=2000, random_state=42, gamma_threshold=0.001,
               passes=10, update_every=1,
               alpha=alpha,eta = beta)

test_corpus_bow = df[df['div']=='test']['corpus'].to_numpy()
test_res = lda_result[test_corpus_bow]

pred=[]
for x in test_res:
    x={k[0]:k[1] for k in x}
    pred.append(max(x,key=x.get) )

y_true = df[df['div']=='test']['label'] 
y_pred = pred
q_metrics(y_true, y_pred)


# evaluate model using Topic Coherence score
cm_lda = CoherenceModel(model=lda_result,
                          dictionary=dictionary, 
                          corpus=df[(df['div']=='train')]['corpus'], 
                          texts=df[df['div']=='train']['tok'].to_numpy(), 
                          coherence='c_v')

# get coherence value
coherence_lda = cm_lda.get_coherence()
    
print('coherence_lda:', coherence_lda)



purity_score: 0.5876288659793815
NMI: 0.2916190058556574
coherence_lda: 0.5133260302092029


In [31]:
pred_test=[]
for x in test_res:
    x={k[0]:k[1] for k in x}
    pred_test.append(max(x,key=x.get) )
    
temp = pd.DataFrame()
temp['y_true'] = y_true
temp['y_pred'] = pred_test
for i in range(TOPICS):
    print(i,'\t',Counter(temp[temp['y_pred']==i]['y_true']))

0 	 Counter({'tamin': 13})
1 	 Counter({'tamin': 8, 'insurance': 6, 'person': 2, 'health': 1})
2 	 Counter({'insurance': 3, 'health': 3, 'tamin': 2})
3 	 Counter({'insurance': 4, 'health': 1})
4 	 Counter({'insurance': 4, 'tamin': 2, 'health': 1})
5 	 Counter({'tamin': 1})
6 	 Counter({'health': 14, 'insurance': 9, 'person': 5, 'tamin': 5})
7 	 Counter({'person': 10, 'insurance': 3})


In [32]:
topic_words=[]
for i in range(TOPICS):
    tt = lda_result.get_topic_terms(i,20)
    topic_words.append([dictionary[pair[0]] for pair in tt])
df_topwords=pd.DataFrame(topic_words)
# df_topwords=df_topwords.T
df_topwords

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,بازنشستگان,تجمع,اعتراضی,اجتماعی,تامین,اعتراض,اهواز,یکشنبه,امینی,مهسا,سراسری,اردیبهشت,معیشتی,اعتصابات,اعتراضات,راهپیمایی,خرداد,وضعیت,اداره,شوش
1,بیمه,شرکت,کار,بازنشستگی,سال,دولت,افزایش,سلامت,پرداخت,صندوق,قانون,حقوق,رییس,رایگان,وزارت,درمانی,کشور,مدیر,پوشش,قرار
2,بیمه,ایران,کار,کشور,بانک,سرمایه,مالیات,ایرانی,زندگی,حقوق,سیستم,هستن,بازار,فروش,اقتصاد,پایین,آینده,نیروی,داره,دلار
3,بیمه,عمر,نمیشه,ایران,برو,زده,مسلح,بابا,نیروهای,خوبه,دوستان,مال,اسم,داره,عکس,توییت,کدوم,ازکی,خدا,امام
4,بیمه,کنه,باشه,پول,بده,داره,میکنه,بشه,کار,کاربلد,دنبال,آدم,اینه,بگیره,نداره,دوست,بیاد,دست,مشکل,پدر
5,تامین,اجتماعی,سازمان,حقوق,کارگر,بازنشسته,ملی,اسلامی,جمهوری,تورم,انقلاب,دولت,فقر,حمایت,هزینه,ملت,صندوق,شغل,خیابان,کف
6,بیمه,تکمیلی,میشه,هزینه,پول,کار,تومن,سال,داره,بیمارستان,ماه,دکتر,حساب,دارن,سابقه,میده,میکنن,حقوق,درمان,رفتم
7,بیمه,ماشین,بدنه,پرداخت,نامه,خسارت,شخص,ثالث,قیمت,خودرو,شکایت,شرکت,تصادف,ثبت,زنگ,مرکزی,سایت,زد,سال,مسیولیت


In [33]:
topic_name={0:'tamin',1:'tamin',2:'insurance',3:'insurance',
            4:'insurance',5:'tamin',6:'health',7:'person'}

y_true = df[df['div']=='test']['label'].to_list()
y_pred = [*map(topic_name.get, pred)]

q_metrics(y_true, pred)
print(classification_report(y_true,y_pred))

purity_score: 0.5876288659793815
NMI: 0.2916190058556574
              precision    recall  f1-score   support

      health       0.42      0.70      0.53        20
   insurance       0.55      0.38      0.45        29
      person       0.77      0.59      0.67        17
       tamin       0.71      0.71      0.71        31

    accuracy                           0.59        97
   macro avg       0.61      0.59      0.59        97
weighted avg       0.61      0.59      0.59        97

